# Load từ drive
- Folder ảnh
- File Label

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/NCKH/images.zip -d /content/images

Archive:  /content/drive/MyDrive/NCKH/images.zip
   creating: /content/images/images/
  inflating: /content/images/images/AnhB6_11_cell_164.jpg  
  inflating: /content/images/images/AnhB6_11_cell_176.jpg  
  inflating: /content/images/images/AnhB6_11_cell_185.jpg  
  inflating: /content/images/images/AnhB6_11_cell_75.jpg  
  inflating: /content/images/images/AnhB6_11_cell_88.jpg  
  inflating: /content/images/images/AnhB6_11_cell_90.jpg  
  inflating: /content/images/images/AnhB6_12_cell_99.jpg  
  inflating: /content/images/images/AnhB6_135_cell_0.jpg  
  inflating: /content/images/images/AnhB6_135_cell_1.jpg  
  inflating: /content/images/images/AnhB6_135_cell_10.jpg  
  inflating: /content/images/images/AnhB6_135_cell_11.jpg  
  inflating: /content/images/images/AnhB6_135_cell_12.jpg  
  inflating: /content/images/images/AnhB6_135_cell_13.jpg  
  inflating: /content/images/images/AnhB6_135_cell_14.jpg  
  inflating: /content/images/images/AnhB6_135_cell_15.jpg  
  inflating: /conten

In [3]:
!cp /content/drive/MyDrive/NCKH/labels.json /content/labels.json

In [ ]:
import os, gc, json, random, warnings
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report, average_precision_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import convnext_base, ConvNeXt_Base_Weights

warnings.filterwarnings("ignore")

# ====================== CONFIG ======================
IMG_DIR = '/content/images/images'
LABEL_FILE = '/content/labels.json'
IMG_SIZE, BATCH_SIZE, EPOCHS, SEED = 224, 32, 10, 42
NUM_CLASSES = 9
ACCUMULATION_STEPS = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ====================== LOAD LABELS ======================
with open(LABEL_FILE) as f:
    label_dict = json.load(f)

df = pd.DataFrame([{"filename": k + ".jpg", "labels": v} for k, v in label_dict.items()])

# ====================== SPLIT DATA ======================
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=SEED)
train_df, val_df = train_test_split(train_val_df, test_size=0.176, random_state=SEED)  # 15% val

# ====================== RARE LABELS ======================
label_counts_total = np.sum(np.stack(df["labels"].values), axis=0)
rare_labels = np.where(label_counts_total < 100)[0].tolist()

# ====================== TRANSFORMS ======================
basic_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

aug_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

# ====================== CUTMIX (NOT USED) ======================
def cutmix(image1, label1, image2, label2, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    bbx1, bby1 = np.random.randint(IMG_SIZE // 2), np.random.randint(IMG_SIZE // 2)
    bbx2, bby2 = bbx1 + IMG_SIZE // 2, bby1 + IMG_SIZE // 2
    image1[:, bby1:bby2, bbx1:bbx2] = image2[:, bby1:bby2, bbx1:bbx2]
    label = lam * label1 + (1 - lam) * label2
    return image1, label

# ====================== DATASET ======================
class CellDataset(Dataset):
    def __init__(self, df, img_dir, transform, cutmix=False, rare_labels=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        img = Image.open(img_path).convert("RGB")
        label = torch.tensor(row["labels"], dtype=torch.float32)
        img = self.transform(img)
        return img, label

# ====================== WEIGHTED SAMPLER ======================
def compute_sample_weights(df):
    label_matrix = np.stack(df["labels"].values)
    label_counts = label_matrix.sum(axis=0)
    label_weights = 1.0 / (label_counts + 1e-6)
    sample_weights = (label_matrix * label_weights).sum(axis=1)
    return sample_weights

sample_weights = compute_sample_weights(train_df)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# ====================== DATALOADER ======================
train_ds = CellDataset(train_df, IMG_DIR, aug_transform)
val_ds = CellDataset(val_df, IMG_DIR, basic_transform)
test_ds = CellDataset(test_df, IMG_DIR, basic_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# ====================== MODEL ======================
model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)
num_ftrs = model.classifier[2].in_features
model.classifier[2] = nn.Sequential(
    nn.LayerNorm((num_ftrs,), eps=1e-6),
    nn.Linear(num_ftrs, NUM_CLASSES)
)
model = model.to(DEVICE)

# ====================== LOSS FUNCTION ======================
criterion = nn.BCEWithLogitsLoss()


# (Giữ lại SPA + PR nhưng không dùng)
class SPALoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        targets = targets.float()
        loss = - targets * torch.log(probs + self.eps) - (1 - targets) * torch.log(1 - probs + self.eps)
        return loss.mean()

def pairwise_regularizer(logits, labels):
    probs = torch.sigmoid(logits)
    loss, count = 0.0, 0
    for i in range(probs.size(0)):
        pos_indices = (labels[i] == 1).nonzero(as_tuple=True)[0]
        neg_indices = (labels[i] == 0).nonzero(as_tuple=True)[0]
        for pos_idx in pos_indices:
            for neg_idx in neg_indices:
                diff = probs[i][neg_idx] - probs[i][pos_idx]
                loss += torch.log(1 + torch.exp(diff))
                count += 1
    return loss / (count + 1e-8)

class CombinedSPALoss(nn.Module):
    def __init__(self, lambda_lpr=0.1):
        super().__init__()
        self.spa = SPALoss()
        self.lambda_lpr = lambda_lpr

    def forward(self, logits, targets):
        spa_loss = self.spa(logits, targets)
        lpr_loss = pairwise_regularizer(logits, targets)
        return spa_loss + self.lambda_lpr * lpr_loss

# ====================== OPTIMIZER / SCHEDULER ======================
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# ====================== THRESHOLD TUNING (NOT USED) ======================
def optimal_threshold(y_true, y_pred_probs):
    best_thresholds = []
    for i in range(y_true.shape[1]):
        best_f1, best_thresh = 0, 0.5
        for t in np.linspace(0.1, 0.9, 17):
            f1 = f1_score(y_true[:, i], y_pred_probs[:, i] > t)
            if f1 > best_f1:
                best_f1, best_thresh = f1, t
        best_thresholds.append(best_thresh)
    return np.array(best_thresholds)

def predict_with_thresholds(model, loader, thresholds):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for imgs, _ in tqdm(loader, desc="Predicting"):
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > thresholds[None, :]).astype(int)
            all_preds.append(preds)
    return np.vstack(all_preds)

# ====================== TRAIN / EVAL ======================
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, global_step = 0, 0
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        if (global_step + 1) % ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
        total_loss += loss.item()
        global_step += 1
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Evaluating"):
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.append(preds)
            all_targets.append(labels.numpy())
    return np.vstack(all_preds), np.vstack(all_targets)

# ====================== maPx / maPy ======================
def compute_map_metrics(y_true, y_probs):
    maPx = np.mean([average_precision_score(t, p) for t, p in zip(y_true, y_probs)])
    maPy = average_precision_score(y_true, y_probs, average='macro')
    return maPx, maPy

Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth
100%|██████████| 338M/338M [00:01<00:00, 194MB/s]


In [13]:
import os, gc, json, random, warnings
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report, average_precision_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import convnext_base, ConvNeXt_Base_Weights

warnings.filterwarnings("ignore")

# ====================== CONFIG ======================
IMG_DIR = '/content/images/images'
LABEL_FILE = '/content/labels.json'
IMG_SIZE, BATCH_SIZE, EPOCHS, SEED = 224, 32, 10, 42
NUM_CLASSES = 9
ACCUMULATION_STEPS = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ====================== LOAD LABELS ======================
with open(LABEL_FILE) as f:
    label_dict = json.load(f)

df = pd.DataFrame([{"filename": k + ".jpg", "labels": v} for k, v in label_dict.items()])

# ====================== SPLIT DATA ======================
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=SEED)
train_df, val_df = train_test_split(train_val_df, test_size=0.176, random_state=SEED)  # 15% val

# ====================== RARE LABELS ======================
label_counts_total = np.sum(np.stack(df["labels"].values), axis=0)
rare_labels = np.where(label_counts_total < 100)[0].tolist()

# ====================== TRANSFORMS ======================
basic_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

aug_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])
'''
# ====================== CUTMIX (NOT USED) ======================
def cutmix(image1, label1, image2, label2, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    bbx1, bby1 = np.random.randint(IMG_SIZE // 2), np.random.randint(IMG_SIZE // 2)
    bbx2, bby2 = bbx1 + IMG_SIZE // 2, bby1 + IMG_SIZE // 2
    image1[:, bby1:bby2, bbx1:bbx2] = image2[:, bby1:bby2, bbx1:bbx2]
    label = lam * label1 + (1 - lam) * label2
    return image1, label
'''
# ====================== DATASET ======================
class CellDataset(Dataset):
    def __init__(self, df, img_dir, transform, cutmix=False, rare_labels=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        img = Image.open(img_path).convert("RGB")
        label = torch.tensor(row["labels"], dtype=torch.float32)
        img = self.transform(img)
        return img, label

# ====================== WEIGHTED SAMPLER ======================
def compute_sample_weights(df):
    label_matrix = np.stack(df["labels"].values)
    label_counts = label_matrix.sum(axis=0)
    label_weights = 1.0 / (label_counts + 1e-6)
    sample_weights = (label_matrix * label_weights).sum(axis=1)
    return sample_weights

sample_weights = compute_sample_weights(train_df)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# ====================== DATALOADER ======================
train_ds = CellDataset(train_df, IMG_DIR, aug_transform)
val_ds = CellDataset(val_df, IMG_DIR, basic_transform)
test_ds = CellDataset(test_df, IMG_DIR, basic_transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

"""
# ====================== MODEL ======================
model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)
num_ftrs = model.classifier[2].in_features
model.classifier[2] = nn.Sequential(
    nn.LayerNorm((num_ftrs,), eps=1e-6),
    nn.Linear(num_ftrs, NUM_CLASSES)
)
model = model.to(DEVICE)

# ====================== LOSS FUNCTION ======================
criterion = nn.BCEWithLogitsLoss()




# (Giữ lại SPA + PR nhưng không dùng)
class SPALoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        targets = targets.float()
        loss = - targets * torch.log(probs + self.eps) - (1 - targets) * torch.log(1 - probs + self.eps)
        return loss.mean()

def pairwise_regularizer(logits, labels):
    probs = torch.sigmoid(logits)
    loss, count = 0.0, 0
    for i in range(probs.size(0)):
        pos_indices = (labels[i] == 1).nonzero(as_tuple=True)[0]
        neg_indices = (labels[i] == 0).nonzero(as_tuple=True)[0]
        for pos_idx in pos_indices:
            for neg_idx in neg_indices:
                diff = probs[i][neg_idx] - probs[i][pos_idx]
                loss += torch.log(1 + torch.exp(diff))
                count += 1
    return loss / (count + 1e-8)

class CombinedSPALoss(nn.Module):
    def __init__(self, lambda_lpr=0.1):
        super().__init__()
        self.spa = SPALoss()
        self.lambda_lpr = lambda_lpr

    def forward(self, logits, targets):
        spa_loss = self.spa(logits, targets)
        lpr_loss = pairwise_regularizer(logits, targets)
        return spa_loss + self.lambda_lpr * lpr_loss
"""
# ====================== OPTIMIZER / SCHEDULER ======================
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# ====================== THRESHOLD TUNING (NOT USED) ======================
def optimal_threshold(y_true, y_pred_probs):
    best_thresholds = []
    for i in range(y_true.shape[1]):
        best_f1, best_thresh = 0, 0.5
        for t in np.linspace(0.1, 0.9, 17):
            f1 = f1_score(y_true[:, i], y_pred_probs[:, i] > t)
            if f1 > best_f1:
                best_f1, best_thresh = f1, t
        best_thresholds.append(best_thresh)
    return np.array(best_thresholds)

def predict_with_thresholds(model, loader, thresholds):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for imgs, _ in tqdm(loader, desc="Predicting"):
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > thresholds[None, :]).astype(int)
            all_preds.append(preds)
    return np.vstack(all_preds)

# ====================== TRAIN / EVAL ======================
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, global_step = 0, 0
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        if (global_step + 1) % ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
        total_loss += loss.item()
        global_step += 1
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Evaluating"):
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.append(preds)
            all_targets.append(labels.numpy())
    return np.vstack(all_preds), np.vstack(all_targets)

# ====================== maPx / maPy ======================
def compute_map_metrics(y_true, y_probs):
    maPx = np.mean([average_precision_score(t, p) for t, p in zip(y_true, y_probs)])
    maPy = average_precision_score(y_true, y_probs, average='macro')
    return maPx, maPy

# Backbone

In [14]:
import torch
import torch.nn as nn
import timm
from torchvision.models import (
    convnext_base, resnet50, efficientnet_b3,
    ConvNeXt_Base_Weights, ResNet50_Weights, EfficientNet_B3_Weights
)

from torchvision.models import ViT_B_16_Weights, vit_b_16
from torchvision.models import swin_t, Swin_T_Weights

def get_backbone(name, num_classes):
    if name == "convnext_base":
        model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1)
        num_ftrs = model.classifier[2].in_features
        model.classifier[2] = nn.Sequential(
            nn.LayerNorm((num_ftrs,), eps=1e-6),
            nn.Linear(num_ftrs, num_classes)
        )
    elif name == "convnextv2_base":
        model = timm.create_model("convnextv2_base.fcmae_ft_in1k", pretrained=True, num_classes=NUM_CLASSES)
    elif name == "resnet50":
        model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif name == "efficientnet_b3":
        model = efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)

    # Các mô hình Transformer
    elif name == "vit":
        model = timm.create_model('vit_base_patch16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Linear(num_ftrs, num_classes)
    elif name == "deit":
        model = timm.create_model('deit_base_distilled_patch16_224', pretrained=True, num_classes=num_classes)
    elif name == "swin":
        model = swin_t(weights=Swin_T_Weights.IMAGENET1K_V1)

        # Đảm bảo output của Swin là [B, F]
        # vì swin.forward_features trả ra [B, 7, 7, F]
        # nhưng swin.forward() sẽ tự avgpool thành [B, F] rồi đưa vào head

        num_ftrs = model.head.in_features
        model.head = nn.Linear(num_ftrs, NUM_CLASSES)
    elif name == "vit_b_16":
        model = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
        num_ftrs = model.heads.head.in_features
        model.heads.head = nn.Linear(num_ftrs, NUM_CLASSES)


    else:
        raise ValueError(f"Unsupported backbone: {name}")

    return model.to(DEVICE)

In [15]:
import time
import gc
from sklearn.metrics import f1_score, accuracy_score, classification_report
import torch

#backbones = ["convnext_base", "resnet50", "efficientnet_b3", "vit", "deit", "swin", "vit_b_16"]
backbones = ["convnext_base", "resnet50", "swin", "efficientnet_b3", "vit", "deit","vit_b_16"]
results = []

for backbone_name in backbones:
    print(f"\n===== Training with backbone: {backbone_name} =====")

    model = get_backbone(backbone_name, NUM_CLASSES)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

    start_time = time.time()

    for epoch in range(EPOCHS):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}")
        scheduler.step()

    elapsed_time = time.time() - start_time

    # Save model weights
    model_save_path = f"{backbone_name}_best_model.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Saved model weights to {model_save_path}")


    # Đánh giá trên tập test
    y_pred_probs_test, y_true_test = evaluate(model, test_loader)
    y_pred_binary_test = (y_pred_probs_test > 0.5).astype(int)

    # Tính toán các chỉ số
    f1_macro = f1_score(y_true_test, y_pred_binary_test, average='macro', zero_division=0)
    f1_micro = f1_score(y_true_test, y_pred_binary_test, average='micro', zero_division=0)
    f1_weighted = f1_score(y_true_test, y_pred_binary_test, average='weighted', zero_division=0)
    #acc = accuracy_score(y_true_test, y_pred_binary_test)
    exact_match = (y_true_test == y_pred_binary_test).all(axis=1).mean()
    maPx, maPy = compute_map_metrics(y_true_test, y_pred_probs_test)

    # In classification report
    print(f"\n=== Classification Report for {backbone_name} ===")
    print(classification_report(y_true_test, y_pred_binary_test, zero_division=0))

    # Ghi kết quả vào list
    results.append({
        "backbone": backbone_name,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "f1_weighted": f1_weighted,
        #"acc": acc,
        "exact_match": exact_match,
        "maPx": maPx,
        "maPy": maPy,
        "time": elapsed_time
    })

    # Dọn dẹp bộ nhớ
    del model
    torch.cuda.empty_cache()
    gc.collect()

# Export kết quả ra CSV
df_result = pd.DataFrame(results)
df_result.to_csv("backbone_comparison.csv", index=False)
print("\n===> Saved results to backbone_comparison.csv")


===== Training with backbone: convnext_base =====


Training: 100%|██████████| 77/77 [01:46<00:00,  1.38s/it]


Epoch 1/10, Train Loss: 0.5907


Training: 100%|██████████| 77/77 [01:37<00:00,  1.26s/it]


Epoch 2/10, Train Loss: 0.5185


...

Epoch 9/10, Train Loss: 0.3183


Training: 100%|██████████| 77/77 [01:21<00:00,  1.06s/it]


Epoch 10/10, Train Loss: 0.3087
Saved model weights to vit_b_16_best_model.pth


Evaluating: 100%|██████████| 17/17 [00:06<00:00,  2.80it/s]



=== Classification Report for vit_b_16 ===
              precision    recall  f1-score   support

           0       0.75      0.97      0.85       356
           1       0.71      0.87      0.78       342
           2       0.40      0.40      0.40        30
           3       0.61      0.55      0.58       103
           4       0.43      0.45      0.44       111
           5       0.64      0.50      0.56       141
           6       0.77      0.78      0.78       250
           7       0.67      0.39      0.49        77
           8       0.64      0.77      0.70       191

   micro avg       0.69      0.75      0.72      1601
   macro avg       0.63      0.63      0.62      1601
weighted avg       0.68      0.75      0.71      1601
 samples avg       0.69      0.78      0.69      1601


===> Saved results to backbone_comparison.csv


In [22]:
import torch
import pandas as pd
from sklearn.metrics import classification_report, f1_score, average_precision_score
from tqdm import tqdm
import io # Import the io module
import numpy as np
import gc

print("===== Generating Reports from Saved Models =====")

summarized_reports_list = [] # List to store summarized classification reports

for backbone_name in backbones:
    print(f"\n=== Evaluating {backbone_name} ===")

    # Load the model architecture
    model = get_backbone(backbone_name, NUM_CLASSES)

    # Load the saved weights
    model_save_path = f"{backbone_name}_best_model.pth"
    try:
        model.load_state_dict(torch.load(model_save_path))
        model = model.to(DEVICE)
        print(f"Successfully loaded weights for {backbone_name}")
    except FileNotFoundError:
        print(f"Error: Weights file not found for {backbone_name} at {model_save_path}. Skipping evaluation.")
        continue
    except Exception as e:
        print(f"Error loading weights for {backbone_name}: {e}. Skipping evaluation.")
        continue

    # Evaluate on the test set
    y_pred_probs_test, y_true_test = evaluate(model, test_loader)
    y_pred_binary_test = (y_pred_probs_test > 0.5).astype(int)

    # Calculate the desired metrics
    f1_macro = f1_score(y_true_test, y_pred_binary_test, average='macro', zero_division=0)
    f1_micro = f1_score(y_true_test, y_pred_binary_test, average='micro', zero_division=0)
    f1_weighted = f1_score(y_true_test, y_pred_binary_test, average='weighted', zero_division=0)
    exact_match = (y_true_test == y_pred_binary_test).all(axis=1).mean()
    # Assuming compute_map_metrics is available from a previous cell
    try:
        maPx, maPy = compute_map_metrics(y_true_test, y_pred_probs_test)
    except NameError:
        print("compute_map_metrics not found. Skipping maPx and maPy calculation.")
        maPx, maPy = None, None


    # Store backbone name and calculated metrics
    summarized_reports_list.append({
        "backbone": backbone_name,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "f1_weighted": f1_weighted,
        "exact_match": exact_match,
        "maPx": maPx,
        "maPy": maPy,
        # Note: 'time' metric is not available here as training is not done in this cell
    })

    # Print classification report (optional, can be removed if only CSV is desired)
    print(f"\n=== Classification Report for {backbone_name} ===")
    print(classification_report(y_true_test, y_pred_binary_test, zero_division=0))


    # Clean up model from memory
    del model
    torch.cuda.empty_cache()
    gc.collect()

# Create a DataFrame from the collected summarized reports
df_summarized_reports = pd.DataFrame(summarized_reports_list)

# Save the DataFrame to a CSV file
report_csv_path = "Base_backbone_comparison.csv"
df_summarized_reports.to_csv(report_csv_path, index=False)
print(f"\n===> Saved backbon comparison to {report_csv_path}")

print("\n===== Finished generating Reports =====")

===== Generating Summarized Classification Reports from Saved Models =====

=== Evaluating convnext_base ===
Successfully loaded weights for convnext_base


Evaluating: 100%|██████████| 17/17 [00:06<00:00,  2.83it/s]



=== Classification Report for convnext_base ===
              precision    recall  f1-score   support

           0       0.75      0.93      0.83       356
           1       0.72      0.88      0.79       342
           2       0.66      0.63      0.64        30
           3       0.63      0.61      0.62       103
           4       0.41      0.54      0.47       111
           5       0.60      0.57      0.58       141
           6       0.79      0.77      0.78       250
           7       0.62      0.42      0.50        77
           8       0.64      0.70      0.67       191

   micro avg       0.69      0.76      0.72      1601
   macro avg       0.65      0.67      0.65      1601
weighted avg       0.69      0.76      0.72      1601
 samples avg       0.69      0.79      0.69      1601


=== Evaluating resnet50 ===
Successfully loaded weights for resnet50


Evaluating: 100%|██████████| 17/17 [00:02<00:00,  7.66it/s]



=== Classification Report for resnet50 ===
              precision    recall  f1-score   support

           0       0.74      0.94      0.83       356
           1       0.70      0.88      0.78       342
           2       0.52      0.40      0.45        30
           3       0.63      0.54      0.58       103
           4       0.46      0.57      0.51       111
           5       0.65      0.61      0.63       141
           6       0.81      0.75      0.78       250
           7       0.64      0.32      0.43        77
           8       0.61      0.70      0.65       191

   micro avg       0.68      0.75      0.71      1601
   macro avg       0.64      0.63      0.63      1601
weighted avg       0.68      0.75      0.71      1601
 samples avg       0.68      0.77      0.68      1601


=== Evaluating swin ===
Successfully loaded weights for swin


...


=== Classification Report for vit ===
              precision    recall  f1-score   support

           0       0.75      0.91      0.82       356
           1       0.72      0.79      0.75       342
           2       0.45      0.50      0.48        30
           3       0.62      0.45      0.52       103
           4       0.45      0.53      0.49       111
           5       0.58      0.52      0.55       141
           6       0.78      0.74      0.76       250
           7       0.60      0.31      0.41        77
           8       0.59      0.66      0.62       191

   micro avg       0.67      0.70      0.69      1601
   macro avg       0.62      0.60      0.60      1601
weighted avg       0.67      0.70      0.68      1601
 samples avg       0.69      0.73      0.66      1601


=== Evaluating deit ===
Successfully loaded weights for deit


Evaluating: 100%|██████████| 17/17 [00:06<00:00,  2.75it/s]



=== Classification Report for deit ===
              precision    recall  f1-score   support

           0       0.77      0.91      0.84       356
           1       0.74      0.83      0.78       342
           2       0.60      0.40      0.48        30
           3       0.64      0.52      0.58       103
           4       0.43      0.47      0.45       111
           5       0.66      0.52      0.58       141
           6       0.80      0.80      0.80       250
           7       0.60      0.35      0.44        77
           8       0.61      0.73      0.67       191

   micro avg       0.70      0.73      0.71      1601
   macro avg       0.65      0.61      0.62      1601
weighted avg       0.70      0.73      0.71      1601
 samples avg       0.71      0.76      0.69      1601


=== Evaluating vit_b_16 ===
Successfully loaded weights for vit_b_16


Evaluating: 100%|██████████| 17/17 [00:06<00:00,  2.76it/s]



=== Classification Report for vit_b_16 ===
              precision    recall  f1-score   support

           0       0.75      0.97      0.85       356
           1       0.71      0.87      0.78       342
           2       0.40      0.40      0.40        30
           3       0.61      0.55      0.58       103
           4       0.43      0.45      0.44       111
           5       0.64      0.50      0.56       141
           6       0.77      0.78      0.78       250
           7       0.67      0.39      0.49        77
           8       0.64      0.77      0.70       191

   micro avg       0.69      0.75      0.72      1601
   macro avg       0.63      0.63      0.62      1601
weighted avg       0.68      0.75      0.71      1601
 samples avg       0.69      0.78      0.69      1601


===> Saved backbon comparison to Base_backbone_comparison.csv

===== Finished generating Summarized Classification Reports =====


In [19]:
# Create a zip file containing all .pth files in the /content directory
!zip models_weights.zip /content/*_best_model.pth

# Display a link to download the zip file (this works in Google Colab)
from google.colab import files
files.download('models_weights.zip')

updating: content/convnext_base_best_model.pth (deflated 7%)
updating: content/deit_best_model.pth (deflated 7%)
updating: content/efficientnet_b3_best_model.pth (deflated 8%)
updating: content/resnet50_best_model.pth (deflated 7%)
updating: content/swin_best_model.pth (deflated 8%)
updating: content/vit_b_16_best_model.pth (deflated 7%)
updating: content/vit_best_model.pth (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>